In [7]:
from talib import abstract
import pandas as pd
import numpy as np
from talib.abstract import *
from quantfreedom._typing import (
    plFrame, plSeries, Union
)
df = pd.read_csv('30min.csv', index_col='time')
df.high = 6

In [5]:
def testing(
    func_name: list,
    prices,
    **kwargs,
):
    np.empty()
    func_name = func_name.upper()
    talib_func = getattr(abstract, func_name)
    return talib_func(prices, **kwargs)
    return talib_func



In [9]:
Function('rsi').info

{'name': 'RSI',
 'group': 'Momentum Indicators',
 'display_name': 'Relative Strength Index',
 'function_flags': ['Function has an unstable period'],
 'input_names': OrderedDict([('price', 'close')]),
 'parameters': OrderedDict([('timeperiod', 14)]),
 'output_flags': OrderedDict([('real', ['Line'])]),
 'output_names': ['real']}

In [10]:
Function('macd').info

{'name': 'MACD',
 'group': 'Momentum Indicators',
 'display_name': 'Moving Average Convergence/Divergence',
 'function_flags': None,
 'input_names': OrderedDict([('price', 'close')]),
 'parameters': OrderedDict([('fastperiod', 12),
              ('slowperiod', 26),
              ('signalperiod', 9)]),
 'output_flags': OrderedDict([('macd', ['Line']),
              ('macdsignal', ['Dashed Line']),
              ('macdhist', ['Histogram'])]),
 'output_names': ['macd', 'macdsignal', 'macdhist']}

In [ ]:
str(rsi)

In [ ]:
import numpy as np
# note that all ndarrays must be the same length!
inputs = {
    'open': np.random.random(100),
    'high': np.random.random(100),
    'low': np.random.random(100),
    'close': np.random.random(100),
    'volume': np.random.random(100)
}

In [ ]:
from talib.abstract import *
output = SMA(inputs, timeperiod=25) # calculate on close prices by default
output = SMA(inputs, timeperiod=25, price='open') # calculate on opens
# upper, middle, lower = BBANDS(inputs, 20, 2, 2)
# slowk, slowd = STOCH(inputs, 5, 3, 0, 3, 0) # uses high, low, close by default
# slowk, slowd = STOCH(inputs, 5, 3, 0, 3, 0, prices=['high', 'low', 'open'])

In [ ]:
new_df = pd.DataFrame(inputs)

In [ ]:
SMA(new_df, timeperiod=25).to_numpy() # calculate on close prices by default

In [ ]:
SMA(new_df, timeperiod=25, prices='open').to_numpy() # calculate on close prices by default

In [ ]:
RSI(new_df, timeperiod=25, price='open').to_numpy()

In [ ]:
RSI(new_df, timeperiod=25, price='high').to_numpy()